In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np


print(torch.__version__)

2.9.0


In [44]:
file_path = "../internal_data/input.txt"

with open(file_path, 'r') as f:
    text = f.read()

In [45]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print("".join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [46]:
stoi = {c:i for i,c in enumerate(chars)}
itoi = {i:c for i, c in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda tokens: ''.join([itoi[t] for t in tokens])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [47]:
data = torch.tensor(encode(text), dtype=torch.long)

In [48]:
print(data.dtype)
print(data.ndim)
print(data[:8])

torch.int64
1
tensor([18, 47, 56, 57, 58,  1, 15, 47])


In [49]:
# spliting data into train/ val: 90% for training
n = len(data)
tr_size = int(n *0.9)
train_data = data[:tr_size]
val_data = data[tr_size:]

print(n); print(len(train_data) / n); print(len(val_data)/n)

1115394
0.8999994620734916
0.10000053792650848


In [50]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1:]

for i in range(block_size):
  context = x[:i+1]
  target = y[i]
  print(f"Input: {context} --> target={target}")

Input: tensor([18]) --> target=47
Input: tensor([18, 47]) --> target=56
Input: tensor([18, 47, 56]) --> target=57
Input: tensor([18, 47, 56, 57]) --> target=58
Input: tensor([18, 47, 56, 57, 58]) --> target=1
Input: tensor([18, 47, 56, 57, 58,  1]) --> target=15
Input: tensor([18, 47, 56, 57, 58,  1, 15]) --> target=47
Input: tensor([18, 47, 56, 57, 58,  1, 15, 47]) --> target=58


In [51]:
torch.manual_seed(1332)

batch_size = 4
block_size = batch_size * 2


def get_batch(split : str):
    # generates small batch for X input and y target
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y


In [52]:
xa, ya = get_batch("train")

print(xa)
print(ya)

tensor([[44,  1, 61, 47, 58, 46,  1, 63],
        [27, 44,  1, 53, 59, 56,  1, 45],
        [40, 59, 58,  1, 39,  1, 57, 50],
        [ 1, 61, 46, 39, 58,  6,  1, 53]])
tensor([[ 1, 61, 47, 58, 46,  1, 63, 53],
        [44,  1, 53, 59, 56,  1, 45, 56],
        [59, 58,  1, 39,  1, 57, 50, 47],
        [61, 46, 39, 58,  6,  1, 53,  5]])


In [53]:
print(decode(xa[0, :].tolist()))
print(decode(xa[:, 0].tolist()))
print(xa)

f with y
fOb 
tensor([[44,  1, 61, 47, 58, 46,  1, 63],
        [27, 44,  1, 53, 59, 56,  1, 45],
        [40, 59, 58,  1, 39,  1, 57, 50],
        [ 1, 61, 46, 39, 58,  6,  1, 53]])


In [60]:
print([decode(tokens) for tokens in xa.tolist()])
print([decode(y_tokens) for y_tokens in ya.tolist()])

['f with y', 'Of our g', 'but a sl', ' what, o']
[' with yo', 'f our gr', 'ut a sli', "what, o'"]


In [61]:
torch.manual_seed(1332)
xb, yb = get_batch("train")

print(xb)
print(yb)
print("-"*32)


for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]

        print(f'Input: {context.tolist()} --> target={target}')


tensor([[44,  1, 61, 47, 58, 46,  1, 63],
        [27, 44,  1, 53, 59, 56,  1, 45],
        [40, 59, 58,  1, 39,  1, 57, 50],
        [ 1, 61, 46, 39, 58,  6,  1, 53]])
tensor([[ 1, 61, 47, 58, 46,  1, 63, 53],
        [44,  1, 53, 59, 56,  1, 45, 56],
        [59, 58,  1, 39,  1, 57, 50, 47],
        [61, 46, 39, 58,  6,  1, 53,  5]])
--------------------------------
Input: [44] --> target=1
Input: [44, 1] --> target=61
Input: [44, 1, 61] --> target=47
Input: [44, 1, 61, 47] --> target=58
Input: [44, 1, 61, 47, 58] --> target=46
Input: [44, 1, 61, 47, 58, 46] --> target=1
Input: [44, 1, 61, 47, 58, 46, 1] --> target=63
Input: [44, 1, 61, 47, 58, 46, 1, 63] --> target=53
Input: [27] --> target=44
Input: [27, 44] --> target=1
Input: [27, 44, 1] --> target=53
Input: [27, 44, 1, 53] --> target=59
Input: [27, 44, 1, 53, 59] --> target=56
Input: [27, 44, 1, 53, 59, 56] --> target=1
Input: [27, 44, 1, 53, 59, 56, 1] --> target=45
Input: [27, 44, 1, 53, 59, 56, 1, 45] --> target=56
Input: [40

In [11]:
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1332)


class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
                
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)  
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):

        for i in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)

            idx_next = torch.multinomial(probs, num_samples=1)

            idx = torch.cat((idx, idx_next), dim=1)
        
        return idx
        
    
model = BiagramLanguageModel(vocab_size)
out, loss = model(xb, yb)
print(out.shape); 
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(model.generate(idx, 60)[0].tolist()))

torch.Size([32, 65])
tensor(4.6546, grad_fn=<NllLossBackward0>)

javIQVz.k'dLgki,qPmetcpTLlaXV;TIJ&mqNHwMc'CbML
$PeTlrNQNJvTb


In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [13]:
batch_size = 32

epochs = 10000

for i in range(epochs):

    xb, yb = get_batch("train")

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f'Iter: {i+1} -> loss={loss.item()}')

Iter: 10000 -> loss=2.3910069465637207


In [17]:
idx = torch.zeros((1, 1), dtype=torch.long)

print(decode(model.generate(idx, 700)[0].tolist()))


ING'amor
IOR:
Ane; s t
JUDeno irre is l.
I

X RDUS:
Fof s:
The,CINGLLous heeie h Y:
OKI:
OMEE at oithce mes IO,
vodoloonse y CLAmy binfotusak t st fithit kse me ffllds, cit arandg lst m, anes w hay yout nke.
SO, fie wo, founghe iofamy ICllyoo an l tid mony; aigime he.
He, Wharurarun:
RWhifthe, srla he beng.
Ant eroung hinde mins f,
Ange fille

And?
KI to'd ele'e izT:
Swese cowstolulist owise my wa'Thello ke youn winithe at?
Dalis heny bipantzlorioore, ar ar empalintha apureng-hou in the vizALY: n?
Tumecod he bionquke ichee?
S:
BPor t hae owhe'se boutavors t do hise ce-hago; hitso t awhe y spofus' an,
Magomy ghed cre cal l meey allseetis she wathowhapesplis our hatKHesedeovit;

IS:
ANUS: fur 


# Mathematical trick of Attention

In [22]:
torch.manual_seed(1337)
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print('A->', a)
print('\nB->', b)

print('\nC->', c)

A-> tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

B-> tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])

C-> tensor([[12., 10.],
        [12., 10.],
        [12., 10.]])


In [40]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

In [51]:
# x[b, t] = gets mean
xbow = torch.zeros((B, T, C))

for b in range(B):
    for i in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = xprev.mean()

In [52]:
xbow

tensor([[[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0496,  0.0496]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.2338,  0.2338]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [-0.2654, -0.2654]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.4749,  0.4749]]])